# **Capital Cities - Acquiring Data From The Web**

### Content:
1. Introduction
2. Acquiring data from the web
3. Data refining
4. Summary
5. References

### 1. Introduction:
* In this notebook we will use python to aquiere data from a Wikipedia page.  
* We will acquire data on capital cities from the Wikipedia page https://en.wikipedia.org/w/index.php?title=List_of_national_capitals&oldid=943802946.  
* Then, we will refine the data so we can use it in future work.

### 2. Acquiring data from the web:  

#### 2.1. Import librararies:

In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)

#### 2.2. *get_table_from_url()* function:
We will start by defining a function that takes a URL and a *__column name__* then returns a corresponding talbe from the URL.  
The function also sets the *__column_name__* to our index column.

In [2]:
def get_table_from_url (url, column_name):
    
    '''Takes a URL and a column name then returns
       a corresponding talbe as a Pandas dataframe.'''
    
    # Get all tables from the target URL:
    table_raw = pd.read_html(url, header = 0)
    
    # For each table try to create a table with the requested column:
    # (This will return only the table that has the requested column)
    for i in range(len(table_raw)):
        try:
            # Create final dataframe and set "column_name" as index:
            return_df = table_raw[i].set_index(column_name).reset_index()
        except:
            continue
            
    return return_df
    
    

Now we use our *get_table_from_url* function retrive a table of Capital Cities from our URL.

In [3]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_national_capitals&oldid=943802946' # URL being accessed 
col = 'Country' # Target column from table and index column

raw_data = get_table_from_url(url, col)
raw_data.head(5)

,Country,City,Notes
0,Ivory Coast,Abidjan (former capital; still has many govern...,NaN
1,Ivory Coast,Yamoussoukro (official),NaN
2,United Arab Emirates,Abu Dhabi,NaN
3,Nigeria,Abuja,Lagos was the capital from 1914 to 1991.
4,Ghana,Accra,NaN


### 3. Data refining:

#### 3.1. Remove unnecessary data:  
First, let's remove an unnecessary columns from our dataframe and give our columns more meaningful names.

In [4]:
# Clean the "df_wikipage" database and romve unnecessary columns
raw_data.drop(columns = 'Notes', inplace = True)

# Rename coluns to be more meaningful:
raw_data.rename(columns = {'City' : 'Capital City'}, inplace = True)

raw_data.head(5)

,Country,Capital City
0,Ivory Coast,Abidjan (former capital; still has many govern...
1,Ivory Coast,Yamoussoukro (official)
2,United Arab Emirates,Abu Dhabi
3,Nigeria,Abuja
4,Ghana,Accra


In [5]:
# Prints a data report message:
print(\
'The Wikipedia page table includes data on {} capital cities from {} unique countries.'\
      .format(raw_data.shape[0], len(raw_data['Country'].unique()))
     )

The Wikipedia page table includes data on 260 capital cities from 244 unique countries.


Our index column *__"Country"__* should have a unique *__Capital_City__* value.  
We can see that this is not the case and we must furthur refine our data.

#### 3.2. Removing duplicates:
Defining a function that locates duplicated values in our dataframe will be useful throughout this notebook.  
We will define a flexible function that can return the duplicated values or the unique values.

#### 3.2.1.*find_duplicates()* function:  
Takes a dataframe and checks a column for duplicated values.  
Returns a dataframe of the duplicated information if *__duplicates = True__* and unique values information if *__duplicates = False__*.

In [6]:
def find_duplicates(check_df, column_name, duplicates = False):
    
    '''Takes a dataframe and checks a selected column for duplicates
       and returns a dataframe with the duplicated values if "duplicates" 
       is set to "True" or a dataframe unique values if "duplicates" is set to False'''
    
    # Create a dataframe of the counted values of "check_df":
    temp_df = pd.DataFrame(check_df[column_name].value_counts()).reset_index()
    
    # Crate a list of unique values in "check_df":
    duplicate_list = check_df[column_name].values
    
    # Keep only duplicated values in the dataframe:
    if duplicates == True:

        for duplicate_value in duplicate_list:
            temp_df = temp_df[temp_df[column_name] > 1]
        # Report amount of duplicated values message:
        print('A total of {} duplicated values found in the "{}" column.'.format(\
                len(temp_df), column_name))
    
    # Keep only unique values in the dataframe:
    else:
        for duplicate_value in duplicate_list:
            temp_df = temp_df[temp_df[column_name] == 1]        
    
    # Create a final dataframe to be returned:
    return_df = pd.DataFrame()
    for country in temp_df['index']:
        return_df = return_df.append(check_df.loc[check_df[column_name] == country])
    return_df.reset_index(drop = True, inplace = True)

    
    
    return return_df

#### 3.2.2. Calling the *find_duplicates()* function on our dataframe:

In [7]:
# Create a dataframe with all the duplicates in our raw_data:
duplicates_data = raw_data.copy()
duplicates_data = find_duplicates(duplicates_data,'Country', duplicates = True)

duplicates_data

A total of 15 duplicated values found in the "Country" column.


,Country,Capital City
0,South Africa,Bloemfontein (judicial)
1,South Africa,Cape Town (legislative)
2,South Africa,Pretoria (executive)
3,Bolivia,La Paz (administrative)
4,Bolivia,Sucre (constitutional)
5,Malaysia,"Kuala Lumpur (official, legislative and royal)"
6,Malaysia,Putrajaya (administrative and judicial)
7,Montserrat,Brades Estate (de facto)
8,Montserrat,Plymouth (official)
9,Sri Lanka,"Colombo (executive, judicial)"


In the *__duplicates_data__* dataframe we can see that there are different types of defenitions of capital cities, for example: "official", "legislative", "de facto", etc.  
We will keep the capital cities that are either "official" or "legislative", and we will keep "offical" over "legislative" when both are avilable.

#### 3.2.3 *find_and_keep()* function:
This function takes a *__dataframe__*, *__column_name__* and two search values, "official" and "legislative" in our case.  
We are flexible with the search values inserted into the function, and it is not mendatory to insert two search values.

In [8]:
def find_and_keep (dataframe, column_name, search1, search2 = None):
    
    '''Takes a dataframe and finds up to two search values in a selected column, then creates
       a new dataframe that inclueds rows that contains at least one of the search values.'''
    
    # Function setup:
    return_df = pd.DataFrame() # Crate a new dataframe
    columns = dataframe.columns # Easy access to original column names
    idex_col = dataframe[columns[0]] # Set an index column to maintain unique values
    unique_values = [] # A list to compare existing values in new dataframe
    
    
    # Find the rows containing at least one of the search values:
    for i,v in enumerate(idex_col):
        
        value = dataframe[column_name][i] # Search this value for search1 and search2
        search1_position = value.find(search1) # Find "search1"
        
        # Make "search2" optional:
        if search2 != None:
            search2_position = value.find(search2) # Find "search2"
        else:
            search2_position = -1

        
        # Continue if unique value already appended into new dataframe:
        if v in unique_values:
            continue
        
        # Append relevant rows to the new dataframe:
        elif search1_position != -1 or search2_position != -1:
            unique_values.append(v)
            return_df = return_df.append(dataframe[dataframe[column_name] == value])

    # Reset the index of the new dataframe:
    return_df.reset_index(drop = True, inplace = True)
    
    return return_df
    

#### 3.2.4. Calling the *find_and_keep()* function on *duplicates_data*:

In [9]:
# Remove duplicates from "duplicates_data":
no_duplicates = find_and_keep(duplicates_data, 'Capital City', 'official', 'legislative')

no_duplicates

,Country,Capital City
0,South Africa,Cape Town (legislative)
1,Malaysia,"Kuala Lumpur (official, legislative and royal)"
2,Montserrat,Plymouth (official)
3,Sri Lanka,Sri Jayawardenepura Kotte (official)
4,Tanzania,"Dodoma (official, legislative)"
5,Ivory Coast,Yamoussoukro (official)
6,Montenegro,Podgorica (official)
7,Netherlands,Amsterdam (official)
8,Benin,Porto-Novo (official)
9,Chile,Santiago (official)


#### 3.2.5. Final dataframe:

Let's call the "find_duplicates" function again, but this time we will set "duplicates" to "Flase" so we will get only countreis with a unique *__Capital City__* value.  
We will also append the *__no_duplicates()__* countreis back in to our dataframe with unique values.

In [10]:
# Create final dataframe:
df_capital = raw_data.copy()

# Get all unique values from the original dataframe:
df_capital = find_duplicates(df_capital,'Country', duplicates = False)

# Append "no_duplicates" to new dataframe:
df_capital = df_capital.append(no_duplicates)

# Reset index:
df_capital.reset_index(drop = True, inplace = True)

df_capital

,Country,Capital City
0,Uruguay,Montevideo
1,Denmark,Copenhagen
2,Costa Rica,San José
3,Belarus,Minsk
4,Bermuda,Hamilton
5,Guinea-Bissau,Bissau
6,North Macedonia,Skopje
7,France,Paris
8,United Kingdom,London
9,Lebanon,Beirut


#### 3.2.6. Removing unnecessary bracketed information:  
In our final step we will clean the string in each cell so it does not have any excess bracketed infromation.

In [11]:
def clear_brackets (dataframe):
    
    '''Takes a dataframe and checks each column for values that have
       bracketed information at the end of each value and removes it.
       (The function skips any values that have bracketed information
       not located at the end of a value)'''
     
    # Get a lit of columns from a dataframe and iterate trhogh them:
    columns = dataframe.columns
    for column in columns:
        for i in range (len(dataframe[column])):
            value = dataframe[column][i] # The value being checked
            cut = value.find(' (') # Find the position of the first bracket
            
            # If located brackets in a value,
            # attempt to remove bracketed information: 
            if cut != -1:
                
                # Bracketed information not located at end of value is skipped:
                save = value.find(') ') # Searching for a space after the brackets
                if save != -1:
                    continue
                    
                # Remove bracketed information that is located at end of value:    
                else:
                    dataframe.loc[dataframe[column].apply(lambda x:\
                    dataframe[column][i] in x) == True, column]\
                    = dataframe[column].str.slice(0,cut)
                    
    return dataframe

In [12]:
df_capital = clear_brackets(df_capital)
df_capital

,Country,Capital City
0,Uruguay,Montevideo
1,Denmark,Copenhagen
2,Costa Rica,San José
3,Belarus,Minsk
4,Bermuda,Hamilton
5,Guinea-Bissau,Bissau
6,North Macedonia,Skopje
7,France,Paris
8,United Kingdom,London
9,Lebanon,Beirut


In [14]:
# Print a report message:
print(\
'The final dataframe of "Capital Cities" has a total of {} \
unique values for {} countries.'\
      .format(df_capital.shape[0],df_capital.shape[0])
     )

The final dataframe of "Capital Cities" has a total of 240 unique values for 240 countries.


### 4. Summary:

Overall, in this notebook we:
1. Acquired data from a Wikipedia page.
2. Define functions that process our dataframe, including:  
    2.1. Finding duplicated data entries.  
    2.2. Removing duplicated values by following a rule we set.  
    2.3. Removing unwanted bracketed information in our data.
3. Refined our data so it can be used in future work.    

__Thank you for taking your time to read through this!__

### 5. References:

Wikipedia contributors. (2019, December). *List of national capitals*. Retrieved March 04, 2020, from Wikipedia: https://en.wikipedia.org/w/index.php?title=List_of_national_capitals&oldid=943802946